# Retrieve EKs collected by TPM PCR tool
Since the `tpm2-algtest` success with retrieving EK is not great, we can compensate using results from `TPM_PCR` measurements.

The only way to get all the EKs is to parse all `*.txt` files, check if they can be parsed via XML parser, because each of those results is a xml file in disguise. If we are successfull we can further check for TPM firmware version and check if given result contains EK values.

As to not do it several times over, the output will be saved conveniently in one `tpm-pcr-metadata.json`

In [1]:
tpm_pcr_path = '/home/tjaros/storage/research/tpm/pcr_windows_results'

In [2]:

import os 

def walk(current_dir, depth):
    """
    Tries to find all the paths for measurement folders

    Sentinel of the recursion is finding a txt file 

    :param current_dir: the directory we process now
    :return: list of paths to valid measurement folders
    """
    predicate = lambda x: x.is_file() and '.txt' in x.name

    scan = list(os.scandir(current_dir))

    if any([predicate(entry) for entry in scan]):
        return [current_dir]

    if depth <= 0:
        return []

    result = []
    for entry in scan:
        if entry.is_dir():
            result += walk(entry.path, depth - 1)

    return result

paths = walk(tpm_pcr_path, 10)

Now we have a list of directories which contain `*.txt` file, now we need to find one file which we are able to parse via XML parser, and contains EK.

In [3]:
len(paths)

688

In [4]:
from xml.etree import ElementTree as ET
import re
measurements = {}
for path in paths:
    scan = list(os.scandir(path))
    for entry in scan:
        try:
            tree = ET.parse(entry.path)
        except Exception: # Actually throws ParseError
            continue
        
        # Get root and retrieve EK
        root = tree.getroot() 
        ek = root.find('EK')

        # If there is no EK, we continue
        if ek is None:
            continue
        ek = ek.text

        # If there is EK, we assume there is RSK
        rsk = root.find('RSK')
        assert rsk is not None
        rsk = rsk.text

        # And there MUST be firmware version
        fw = root.findall('.//FirmwareVersion')
        if fw == []:
            print(entry.path)
            continue
        fw = fw[0].text
        fw = fw.replace('  ', ' ')

        tpm = root.findall('.//TPM')
        tpm_version = None
        tpm_revision = None
        
        if tpm == []:
            print(f"No <TPM> tag found {entry.path}")
        else:
            if (res := re.search(r"TPM-Version:(\d+\.\d+).*Revision:(\d+\.\d+)", tpm[0].text)) is not None:
                tpm_version = res.group(1)
                tpm_revision = res.group(2)
            

        measurements.setdefault(fw, {'TPM Version': tpm_version, 'TPM Revision': tpm_revision,'EK':[], 'RSK': [], 'paths': []})

        
        if ek not in measurements[fw]['EK'] or rsk not in measurements[fw]['RSK']:
            measurements[fw]['EK'].append(ek)
            measurements[fw]['RSK'].append(rsk)
            measurements[fw]['paths'].append(entry.path)
        
len(measurements)

51

In [5]:
sum = 0
for values in measurements.values():
    sum += len(values['EK'])   
sum

198

In [6]:
import json
with open('./tpm-pcr-metadata.json', 'w') as f:
    json.dump(measurements, f, indent=4)